# Use Case 1: Selecting and enriching specified buildings

## Get existing data set
In this example we used data from the [GEOportal.NRW](https://www.geoportal.nrw/?activetab=map).

### Importing need libraries

In [1]:
from citydpc import Dataset
from citydpc.core.input.citygmlInput import load_buildings_from_xml_file
from citydpc.tools import cityATB
import glob

### Loading all files

In [ ]:
# create empty dataset
dataset = Dataset()

# get all citygml files in the folder
files = glob.glob('files/3d-gm_lod2_kacheln/*.gml')
for file in files:
    load_buildings_from_xml_file(dataset, file)
print(f"The dataset hast {len(dataset)} buildings")

## Searching using CityATB

### Searching using addresses

In [ ]:
streetNameDataset = cityATB.search_dataset(dataset, addressRestriciton={"thoroughfareName": "Stakenholt"})
len(streetNameDataset)

### Searching using border coordinates

In [ ]:
# you can also create a coordinate border using the borderCoordinates argument
borderCoorDataset = cityATB.search_dataset(dataset, borderCoordinates=[[360057.31, 5706881.64], [360057.31, 5706267.41], [359792.94, 5706267.41], [359792.94, 5706881.64]])
len(borderCoorDataset)

### Combined search

In [ ]:
# you can also do both operations at the same time (and even in the load_buildings_from_xml_file function)
combinedSearchDataset = cityATB.search_dataset(
    dataset,
    borderCoordinates=[[360057.31, 5706881.64], [360057.31, 5706267.41], [359792.94, 5706267.41], [359792.94, 5706881.64]],
    addressRestriciton={"thoroughfareName": "Stakenholt"},
)
len(combinedSearchDataset)

### Enriching with semantic information

In [8]:
# set the year of construction and the number of storeys above ground
# for every building in the dataset
for building in combinedSearchDataset.get_building_list():
    building.yearOfConstruction = 1970
    building.storeysAboveGround = 3

## Move buildings to different positon

In [9]:
from citydpc.tools import cityGTV

# transform dataset (only moving with dataset in this case)
movedDataset = cityGTV.transform_dataset(
    combinedSearchDataset,
    "epsg:5555",
    "epsg:5555",
    (324219.701, 5641681.61),
    (360133.335, 5706839.754),
    rotAngle=0,
    eleChange=0,
)

## Save dataset to CityGML file

In [12]:
from citydpc.core.output.citygmlOutput import write_citygml_file

# set SRS of the dataset
movedDataset.srsName = "epsg:5555"
write_citygml_file(movedDataset, "reducedEnrichedMovedDataset.gml")